In [4]:
import pandas as pd
import os

export_path = r"C:\Users\User\Downloads\COB DATA\2025\10.October\10.23.25\Export"
import_path = r"C:\Users\User\Downloads\COB DATA\2025\10.October\10.23.25\COB\Data"
os.makedirs(export_path, exist_ok=True)


In [5]:
#INVOICE ON Arrangement

im_filename_inv = "DMS-Invoice-on.xlsx"
full_im_path_inv = os.path.join(import_path, im_filename_inv)

df1 = pd.read_excel(full_im_path_inv)

dfc = df1[['Invoice Date', 'Sold To Customer Number',
'Sold To Customer Name', 'Product Code', 'Product/Item Description',
'Total Item amount with Tax and Discount', 'Invoice Item Type']]
inv = dfc[dfc['Invoice Item Type'] != 'ITM_SALES_TAX']

inv_f = inv
inv_f.insert(6,'BO',0)
inv_f.insert(7,'FG',0)
inv_f['Total Item amount with Tax and Discount'] = inv_f['Total Item amount with Tax and Discount'].fillna(0)
inv_f = inv_f.drop('Invoice Item Type', axis=1)


ex_filename_inv = "Invoices.xlsx"
full_path_inv = os.path.join(export_path, ex_filename_inv)

export_inv = inv_f
export_inv = export_inv.drop(['BO', 'FG'], axis=1)
export_inv = export_inv.sort_values(by=['Invoice Date', 'Sold To Customer Number'])
export_inv.to_excel(full_path_inv, index=False)

c:\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\User\AppData\Local\Temp\ipykernel_12896\258991540.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inv_f['Total Item amount with Tax and Discount'] = inv_f['Total Item amount with Tax and Discount'].fillna(0)


In [7]:
#CUSTOMER RETURNS Arrangement

im_filename_ret = "DMS-Customer Returns-on.xlsx"
full_im_path_ret = os.path.join(import_path, im_filename_ret)

df2 = pd.read_excel(full_im_path_ret)

dfc2 = df2[['Customer Return Date', 'Sold To Customer Number', 'Sold To Customer Name', 
'Product Code', 'Product Description', 'Facility Name', 'Estimated Product Return Amount']]
dfc2['Estimated Product Return Amount'].fillna(0, inplace=True)
dfc2.insert(5,'Total Item amount with Tax and Discount', 0)
dfc2.insert(8,'with vat', 0)
dfc2['with vat'] = dfc2['Estimated Product Return Amount'] * 1.12

cust_ret = dfc2.pivot_table(index=['Customer Return Date', 'Sold To Customer Number', 'Sold To Customer Name', 
'Product Code', 'Product Description', 'Total Item amount with Tax and Discount'], columns='Facility Name', values='with vat', aggfunc=sum)

ret_f = cust_ret.reset_index()
ret_f['BO'] = ret_f['BO'].fillna(0)
ret_f['FG'] = ret_f['FG'].fillna(0)

C:\Users\User\AppData\Local\Temp\ipykernel_12896\3676257053.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dfc2['Estimated Product Return Amount'].fillna(0, inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_12896\3676257053.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc2['Estimated Product Return Amount'].fillna(0, inpla

In [8]:
#COMBINE INVOICED AND RETURNS to NET INVOICED

inv_f.rename(columns={
'Invoice Date': 'DATE',
'Sold To Customer Number': 'ACCOUNT CODE',
'Sold To Customer Name': 'ACCOUNT NAME',
'Product Code': 'SKU CODE',
'Product/Item Description': 'SKU NAME',
'Total Item amount with Tax and Discount': 'SERVED INVOICE',
'BO': 'BAD RETURNS',
'FG': 'GOOD RETURNS'
}, inplace=True)

ret_f.rename(columns={
'Customer Return Date': 'DATE',
'Sold To Customer Number': 'ACCOUNT CODE', 
'Sold To Customer Name': 'ACCOUNT NAME',
'Product Code': 'SKU CODE', 
'Product Description': 'SKU NAME',
'Total Item amount with Tax and Discount': 'SERVED INVOICE',
'BO': 'BAD RETURNS',
'FG': 'GOOD RETURNS'
}, inplace=True)

net_inv = pd.concat([inv_f, ret_f], axis=0)

ex_filename_net_inv = "Net_Invoiced.xlsx"
full_path_net_inv = os.path.join(export_path, ex_filename_net_inv)

export_net_inv = net_inv
export_net_inv.to_excel(full_path_net_inv, index=False)

In [9]:
#SALES ORDERS

im_filename_so = "DMS-Sales Order-on.xlsx"
full_im_path_so = os.path.join(import_path, im_filename_so)

df3 = pd.read_excel(full_im_path_so)

dfc3 = df3[['Last Modified Date', 'Sold To Customer number', 'Sold To Customer Name',
 'Product Code', 'Product Description', 'Total Product Amount', 'SO status']]
so_par = dfc3[dfc3['SO status'] == 'Invoiced']

so_par['Total Product Amount'].fillna(0, inplace=True)
so_par.insert(6,'with vat', 0)
so_par['with vat'] = so_par['Total Product Amount'] * 1.12

so_f = so_par
so_f = so_f.drop(['Total Product Amount', 'SO status'], axis=1)

ex_filename_so = "Sales_Orders.xlsx"
full_path_so = os.path.join(export_path, ex_filename_so)

export_so = so_f
export_so.to_excel(full_path_so, index=False)

c:\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\User\AppData\Local\Temp\ipykernel_12896\2300135604.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  so_par['Total Product Amount'].fillna(0, inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_12896\2300135604.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF